In [15]:
from Bio import SeqIO
from Bio import Entrez
import ssl
import certifi
from tqdm.notebook import tqdm

Entrez.email = "first.last@email.com"

# Use certifi's CA bundle for SSL verification
ssl_context = ssl.create_default_context(cafile=certifi.where())

# Configure Entrez to use the updated SSL context
import urllib.request
opener = urllib.request.build_opener(urllib.request.HTTPSHandler(context=ssl_context))
urllib.request.install_opener(opener)

# File paths
input_fasta = "SILVA_input.fasta" # formerly called "SILVA_138.2_SSURef_NR99_tax_silva.fasta"
output_16S_fasta = "16S_SILVA_sequences.fasta"  # Output file for 16S rRNA sequences
output_18S_fasta = "18S_SILVA_sequences.fasta"  # Output file for 18S rRNA sequences

# Function to fetch gene identity from NCBI and validate sequence length
def fetch_gene_info_and_check_length(accession, sequence_length):
    try:
        # Query NCBI for the given accession number
        handle = Entrez.efetch(db="nucleotide", id=accession, retmode="xml")
        records = Entrez.read(handle)
        handle.close()
        
        # Look for rRNA features in the returned records
        for record in records:
            for feature in record["GBSeq_feature-table"]:
                if feature["GBFeature_key"] == "rRNA":
                    # Check if "16S" or "18S" is mentioned in the qualifiers
                    for qualifier in feature["GBFeature_quals"]:
                        if "16S" in qualifier["GBQualifier_value"]:
                            # Validate length for 16S
                            if 1200 < sequence_length < 2000:
                                return "16S"
                        elif "18S" in qualifier["GBQualifier_value"]:
                            # Validate length for 18S
                            if 1500 < sequence_length < 2400:
                                return "18S"
        return None  # No valid 16S/18S found or length not in range
    except Exception as e:
        print(f"Error fetching info for accession {accession}: {e}")
        return None

# Function to find the last processed accession ID in an output file
def find_last_processed_id(output_file):
    try:
        with open(output_file, "r") as file:
            records = list(SeqIO.parse(file, "fasta"))
            if records:
                return records[-1].id.split(".")[0]  # Return the accession without the version
        return None
    except FileNotFoundError:
        return None

# Find the last processed accession IDs
last_16S_id = find_last_processed_id(output_16S_fasta)
last_18S_id = find_last_processed_id(output_18S_fasta)

# Determine the most recent accession ID to resume from
last_processed_id = None
if last_16S_id and last_18S_id:
    last_processed_id = max(last_16S_id, last_18S_id)
elif last_16S_id:
    last_processed_id = last_16S_id
elif last_18S_id:
    last_processed_id = last_18S_id

# Open the input file and output files (append mode for output)
with open(input_fasta, "r") as infile, \
     open(output_16S_fasta, "a") as outfile_16S, \
     open(output_18S_fasta, "a") as outfile_18S:

    # Parse the input FASTA file with tqdm for progress tracking
    skip = True if last_processed_id else False  # Skip until we find the last processed ID
    count_16S = 0
    count_18S = 0
    count_fail = 0

    for record in tqdm(SeqIO.parse(infile, "fasta"), desc="Processing sequences", unit="record", ncols=400):
        accession = record.id.split(".")[0]
        sequence_length = len(record.seq)

        # Skip records until we find the last processed ID
        if skip:
            if accession == last_processed_id:
                skip = False  # Resume processing after this record
            continue

        # Fetch gene information and validate length
        gene_type = fetch_gene_info_and_check_length(accession, sequence_length)

        # Write valid sequences to the appropriate output file
        if gene_type == "16S":
            SeqIO.write(record, outfile_16S, "fasta")
            count_16S += 1
        elif gene_type == "18S":
            SeqIO.write(record, outfile_18S, "fasta")
            count_18S += 1
        else:
            count_fail += 1

# Print summary of results
print(f"\nExtraction complete:")
print(f"16S rRNA sequences written in this session: {count_16S}")
print(f"18S rRNA sequences written in this session: {count_18S}")
print(f"??? rRNA sequences ignored in this session: {count_fail}")

Processing sequences: 0record [00:00, ?record/s]

KeyboardInterrupt: 